# Figure panel 3

In [ ]:
options(warn=-1)

In [ ]:
library_load <- suppressMessages(
    
    suppressWarnings(
        
        list(
        
            # Seurat 
            library(Seurat), 

            # TradeSeq
            library(CellChat),

            # Data 
            library(tidyverse), 
            library(data.table), 

            # miloR
            library(miloR), 
            library(ggbeeswarm), 

            # Plotting 
            library(ggplot2), 
            library(patchwork), 
            library(cowplot), 
            library(ComplexHeatmap), 

            # Pyhton compatibility
            library(reticulate)

        )
    )
)

In [ ]:
# Configure reticulate 
# use_condaenv(condaenv="p.3.10.16-FD20200109SPLENO", conda="/nobackup/peer/fdeckert/miniconda3/bin/conda", required=NULL)
# py_config()

In [ ]:
random_seed <- 42
set.seed(random_seed)

In [ ]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [ ]:
# Source
source("bin/so_pl.R")
source("bin/dp_feature.R")
source("bin/dea_pl.R")
source("bin/tradeseq_pl.R")
source("bin/tradeseq_pp.R")
source("bin/cci_pp.R")
source("bin/cci_pl.R")
source("bin/reactome.R")

In [ ]:
# Plotting Theme
source("plotting_global.R")
ggplot2::theme_set(theme_global_set(size_select=4)) # From project global source()

# Load reference data 

In [ ]:
tf <- read.table("/research/peer/fdeckert/reference/animaltfdb/Mus_musculus_TF.txt")[[2]]

lr <- CellChat::CellChatDB.mouse[[1]] %>% dplyr::select(pathway_name, ligand, receptor.symbol, receptor.family, annotation) %>% dplyr::filter(annotation %in% c("Cell-Cell Contact", "Secreted Signaling", "ECM-Receptor")) %>% 
    separate_rows(receptor.symbol, sep=", ") %>% dplyr::distinct() %>%
    separate_rows(ligand, sep=", ") %>% dplyr::distinct() %>% 
    dplyr::rename(pathway=pathway_name, receptor=receptor.symbol, family=receptor.family)

# Import scRNAseq data 

In [ ]:
so <- readRDS("data/scRNAseq/object/pp_1.rds")
so <- NormalizeData(so)

# UMAP leiden celltype low

In [ ]:
dp_1 <- dplot(so, group_by="celltype_low", alpha=1.0, pt_size=0.25, shuffle=TRUE, size_select=4) + 

    scale_color_manual(values=color$celltype_low) + 
    scale_fill_manual(values=color$celltype_low) + 
    guides(color=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0, keyheight=0.25, default.unit="cm")) + 
    theme(
    
        legend.text.align=0
    
    ) 

In [ ]:
pdf("result/figures/1_scRNAseq/panel_3/umap_celltype_low.pdf", width=3.5, height=2.5)

dp_1

dev.off()

# Barplot celltype ratio  

In [ ]:
so$celltype_high <- ifelse(so$celltype_low %in% c("MEP", "Proerythroblast", "Erythroblast"), "E", ifelse(so$celltype_low %in% c("MegP", "GMP", "NeuP", "BasoP", "MastP"), "P", "M"))

In [ ]:
so$sample_group_rep <- so$sample_group
so$sample_group_rep <- factor(so$sample_group_rep, levels=c("IFNAR_fl_Baseline_D0", "IFNAR_fl_LysM_cre_Baseline_D0", "IFNAR_fl_CpG_D1",  "IFNAR_fl_LysM_cre_CpG_D1", "IFNAR_fl_CpG_D3", "IFNAR_fl_LysM_cre_CpG_D3"))

In [ ]:
bp_1 <- ggplot(so@meta.data[so$celltype_high=="E", ], aes(x=sample_group_rep, fill=celltype_low)) + 
    geom_bar(position="fill", width=0.8, color="black", size=0.1) + 
    scale_fill_manual(values=color$celltype_low) + 
    ggtitle("") + xlab("") + ylab("Celltype [ratio]") + 
    facet_wrap(~celltype_high, scales="free", ncol=1) + 
    scale_x_discrete(labels=c("+/+ Ctl", "cre/+ Ctl", "+/+ D1", "cre/+ D1", "+/+ D3", "cre/+ D3")) + 
    theme(axis.text.x=element_text(angle=90, vjust=0.5, hjust=1)) + 
    guides(fill=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0.25, keyheight=0.25, default.unit="cm"))

bp_2 <- ggplot(so@meta.data[so$celltype_high=="P", ], aes(x=sample_group_rep, fill=celltype_low)) + 
    geom_bar(position="fill", width=0.8, color="black", size=0.1) + 
    scale_fill_manual(values=color$celltype_low) + 
    ggtitle("") + xlab("") + ylab("Celltype [ratio]") + 
    facet_wrap(~celltype_high, scales="free", ncol=1) + 
    scale_x_discrete(labels=c("+/+ Ctl", "cre/+ Ctl", "+/+ D1", "cre/+ D1", "+/+ D3", "cre/+ D3")) + 
    theme(axis.text.x=element_text(angle=90, vjust=0.5, hjust=1)) + 
    guides(fill=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0.25, keyheight=0.25, default.unit="cm"))

bp_3 <- ggplot(so@meta.data[so$celltype_high=="M", ], aes(x=sample_group_rep, fill=celltype_low)) + 
    geom_bar(position="fill", width=0.8, color="black", size=0.1) + 
    scale_fill_manual(values=color$celltype_low) + 
    ggtitle("") + xlab("") + ylab("Celltype [ratio]") + 
    facet_wrap(~celltype_high, scales="free", ncol=1) + 
    scale_x_discrete(labels=c("+/+ Ctl", "cre/+ Ctl", "+/+ D1", "cre/+ D1", "+/+ D3", "cre/+ D3")) + 
    theme(axis.text.x=element_text(angle=90, vjust=0.5, hjust=1)) + 
    guides(fill=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0.25, keyheight=0.25, default.unit="cm"))

In [ ]:
pdf("result/figures/1_scRNAseq/panel_3/bp_celltype_low.pdf", width=2, height=4)

gridExtra::grid.arrange(

    bp_1 %>% egg::set_panel_size(., width=unit(6*0.30, "cm"), height=unit(2.0, "cm")),
    bp_2 %>% egg::set_panel_size(., width=unit(6*0.30, "cm"), height=unit(2.0, "cm")),
    bp_3 %>% egg::set_panel_size(., width=unit(6*0.30, "cm"), height=unit(2.0, "cm")), ncol=1
        
)

dev.off()

# Celltype marker genes

In [ ]:
marker_genes=data.frame(

    celltype=c(
        
        "Prog.", "Prog.", "Prog.", "Prog.", "Prog.",

        "NeuP", 
               
        "Baso.", "Baso.", "Baso.", "Baso.", 
               
        "Mast", "Mast", 

        "MegP", "MegP", 
               
        "Erythroid", "Erythroid", "Erythroid", "Erythroid", "Erythroid", "Erythroid", "Erythroid", "Erythroid", "Erythroid", 
               
        "Mono.", "Mono.", "Mono.", "Mono.", 
               
        "RPM", "RPM", "RPM", 
               
        "cDC", "cDC", "cDC", "cDC", "cDC", "cDC", "cDC"
               
    ), 
    
    genes=c(
        
        "Hlf", "Cd34", "Kit", "Meis1", "Spi1",  

        "Elane", 
            
        "Cebpa", "Mcpt8", "Clec12a", "Itga2", 
            
        "Mcpt4", "Cma1", 

        "Pf4", "Itga2b",
            
        "Gata2", "Gata1", "Klf1", "Icam4", "Epor", "Sox6", "Tfrc", "Alas2", "Slc4a1",
            
        "Ly6c2", "Ccr2", "Cx3cr1", "Itgam",
            
        "Adgre1", "Spic", "Cd163",  
            
        "Sirpa", "Flt3", "Xcr1", "Cd8a", "Cd4", "Cd209a", "Ccr7"
    
    )
    
)

In [ ]:
dp_1 <- dp_feature(so, marker_genes$genes, group_by="celltype_low", group_by_order=rev(levels(so$celltype_low)), split=marker_genes$celltype, split_order=unique(marker_genes$celltype), range_max=2.5) + 

    scale_y_discrete(labels=rev(levels(so$celltype_low))) + theme_global_set(4) + 

        theme(
            legend.position="bottom", 
            panel.spacing=unit(0.1, "lines"), 
            axis.text.x=element_text(angle=90, vjust=0.5, hjust=1, face="italic"),
            axis.text.y=element_text(angle=0, vjust=0.5, hjust=1), 
            strip.text.x=element_text(angle=90, vjust=0.5, hjust=0), 
            strip.text=element_text(margin=margin(0.1, 0.1, 0.1, 0.1, "lines")), 
            strip.background=element_rect(fill="transparent", color=NA), 
            legend.key.size=unit(0.25, "cm"), 
            legend.key.height=unit(0.25, "cm"), 
            legend.key.width=unit(0.25, "cm")
            
        )

In [ ]:
pdf("result/figures/1_scRNAseq/panel_3/dp_celltype_low_marker_genes.pdf", width=4.0, height=3.2)

dp_1

dev.off()

# DEA overview 

In [ ]:
adj_pval_thr <- 0.05
log2FC_thr <- 0.25

In [ ]:
contrasts_vec <- c("IFNAR_fl_CpG_D1", "IFNAR_fl_Baseline_D0")
dea_res_1 <- readRDS(paste0("result/dea/scRNAseq/sample_group/", contrasts_vec[1], "_vs_", contrasts_vec[2], ".rds"))

In [ ]:
contrasts_vec <- c("IFNAR_fl_CpG_D3", "IFNAR_fl_Baseline_D0")
dea_res_2 <- readRDS(paste0("result/dea/scRNAseq/sample_group/", contrasts_vec[1], "_vs_", contrasts_vec[2], ".rds"))

In [ ]:
mat_1 <- lapply(dea_res_1, function(x) {x %>% dplyr::filter(p_val_adj <= adj_pval_thr & abs(avg_log2FC)>=log2FC_thr) %>% dplyr::pull(gene) %>% length}) %>% data.frame() %>% t()
mat_2 <- lapply(dea_res_2, function(x) {x %>% dplyr::filter(p_val_adj <= adj_pval_thr & abs(avg_log2FC)>=log2FC_thr) %>% dplyr::pull(gene) %>% length}) %>% data.frame() %>% t()

In [ ]:
mat <- cbind(mat_1, mat_2) %>% data.frame() %>% dplyr::rename(Ctr_vs_D1=X1, Ctr_vs_D3=X2)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_3/hm_dea_cg.pdf", width=1.5, height=3)

draw(dea_count_hm(as.matrix(mat), fontsize_select=2))

dev.off()

# DEA Volcano plots  

## D1 vs Ctl

In [ ]:
contrasts_vec <- c("IFNAR_fl_CpG_D1", "IFNAR_fl_Baseline_D0")
dea_res_1 <- readRDS(paste0("result/dea/scRNAseq/sample_group/", contrasts_vec[1], "_vs_", contrasts_vec[2], ".rds"))

In [ ]:
labels <- list(
    
)

In [ ]:
vp_1 <- lapply(names(dea_res_1), function(i) {

    if(i %in% names(labels)) {

        label <- labels[[i]]
        
    } else {

        label <- NULL
        
    }

    v_pl(dea_res_1[[i]], title=i, log2FC_thr=log2FC_thr , adj_pval_thr=adj_pval_thr, color_pos=color$sample_group[contrasts_vec[1]], color_neg=color$sample_group[contrasts_vec[2]], label=label, top_label=15, label_merge=FALSE, point_size=0.5, label_size=2, aspect_ratio=0.8, y_limit=5) + theme(legend.position="none") + theme_global_set(size_select=4)

}
              )

In [ ]:
pdf("result/figures/1_scRNAseq/panel_3/v_pl_d1_vs_ctl.pdf", width=5*2, height=4*2)

plots <- lapply(vp_1, function(p) egg::set_panel_size(p, width=unit(4, "cm"), height=unit(0.8*4, "cm")))
do.call(gridExtra::grid.arrange, c(plots, ncol=5, nrow=4, newpage=FALSE))

dev.off()

## D3 vs Ctl 

In [ ]:
contrasts_vec <- c("IFNAR_fl_CpG_D3", "IFNAR_fl_Baseline_D0")
dea_res_1 <- readRDS(paste0("result/dea/scRNAseq/sample_group/", contrasts_vec[1], "_vs_", contrasts_vec[2], ".rds"))

In [ ]:
labels <- list(
    
)

In [ ]:
vp_1 <- lapply(names(dea_res_1), function(i) {

    if(i %in% names(labels)) {

        label <- labels[[i]]
        
    } else {

        label <- NULL
        
    }

    v_pl(dea_res_1[[i]], title=i, log2FC_thr=log2FC_thr , adj_pval_thr=adj_pval_thr, color_pos=color$sample_group[contrasts_vec[1]], color_neg=color$sample_group[contrasts_vec[2]], label=label, top_label=15, label_merge=FALSE, point_size=0.5, label_size=2, aspect_ratio=0.8, y_limit=5) + theme(legend.position="none") + theme_global_set(size_select=4)

}
              )

In [ ]:
pdf("result/figures/1_scRNAseq/panel_3/v_pl_d3_vs_ctl.pdf", width=5*2, height=4*2)

plots <- lapply(vp_1, function(p) egg::set_panel_size(p, width=unit(4, "cm"), height=unit(0.8*4, "cm")))
do.call(gridExtra::grid.arrange, c(plots, ncol=5, nrow=4, newpage=FALSE))

dev.off()

# CellChat overview 

In [ ]:
cco_1 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_baseline_d0.rds")
cco_2 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_cpg_d1.rds")
cco_3 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_cpg_d3.rds")

In [ ]:
cco_diff_1 <- list(IFNAR_fl_Baseline_D0=cco_1, IFNAR_fl_CpG_D1=cco_2)
cco_diff_1 <- mergeCellChat(cco_diff_1, add.names=names(cco_diff_1))
cco_diff_1@meta$sample_group <- factor(cco_diff_1@meta$sample_group, levels=c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D1"))

In [ ]:
cco_diff_2 <- list(IFNAR_fl_Baseline_D0=cco_1, IFNAR_fl_CpG_D3=cco_2)
cco_diff_2 <- mergeCellChat(cco_diff_2, add.names=names(cco_diff_2))
cco_diff_2@meta$sample_group <- factor(cco_diff_2@meta$sample_group, levels=c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D3"))

## Interaction count 

In [ ]:
source <- NULL
target <- c("MEP", "Proerythroblast", "Erythroblast")

In [ ]:
pdf("result/figures/1_scRNAseq/panel_3/hm_net_ctr_cg.pdf", width=2.5, height=2.5)

gridExtra::grid.arrange(
    
   hm_net(cellchat=cco_1, slot="count", source=source, target=target, title="Interaction count Ctr", breaks_max=30, fontsize_select=2) %>% ggplotify::as.ggplot()

)

dev.off()

In [ ]:
pdf("result/figures/1_scRNAseq/panel_3/hm_net_d1_cg.pdf", width=2.5, height=2.5)

gridExtra::grid.arrange(
    
   hm_net(cellchat=cco_2, slot="count", source=source, target=target, title="Interaction count D1", breaks_max=30, fontsize_select=2) %>% ggplotify::as.ggplot()

)

dev.off()

In [ ]:
pdf("result/figures/1_scRNAseq/panel_3/hm_net_d3_cg.pdf", width=2.5, height=2.5)

gridExtra::grid.arrange(
    
   hm_net(cellchat=cco_3, slot="count", source=source, target=target, title="Interaction count D3", breaks_max=30, fontsize_select=2) %>% ggplotify::as.ggplot()

)

dev.off()

# CCI DEA 

In [ ]:
# Parameter 
pval_prob_thr <- 0.05
p_val_adj_dea <- 0.05
pathway_name_thr <- c("ICAM", "VCAM", "SIRP", "KIT", "TGFb", "TNF", "CXCL", "CCL")

# CellChat results
lr_res <- read.csv("result/cci/lr_res.csv", row.names=1)

# Add reversed Cell-cell interactions 
db <- CellChat::CellChatDB.mouse

# Impute custom interactions
options(stringsAsFactors=FALSE)
interaction <- read.csv("data/reference/cellchat/interaction.csv", row.names=1, na.strings=c(""))
interaction[is.na(interaction)] <- ""
db$interaction <- rbind(db$interaction, interaction)

# Get cell-cell contact interaction 
db <- CellChat::subsetDB(db, search=c("Cell-Cell Contact"), key="annotation")

# Subset lr results for cell-cell contact forward 
lr_res_ccc_fwd <- lr_res %>% dplyr::filter(lr_res$interaction_name %in% db$interaction$interaction_name)

# Reverse cell-cell contact 
lr_res_ccc_rev <- data.frame(

    source=lr_res_ccc_fwd$target, 
    target=lr_res_ccc_fwd$source, 
    interaction_name=paste0(lr_res_ccc_fwd$receptor_symbol, "_", lr_res_ccc_fwd$ligand_symbol), 
    prob=lr_res_ccc_fwd$prob, 
    pval=lr_res_ccc_fwd$pval, 
    pathway_name=lr_res_ccc_fwd$pathway_name, 
    ligand_symbol=lr_res_ccc_fwd$receptor_symbol, 
    receptor_symbol=lr_res_ccc_fwd$ligand_symbol, 
    sample_group=lr_res_ccc_fwd$sample_group
    
)

# Combine results 
lr_res_ccc <- rbind(lr_res_ccc_fwd, lr_res_ccc_rev)

# Assemble LR results 
lr_res <- rbind(lr_res[!lr_res$interaction_name %in% lr_res_ccc$interaction_name, ], lr_res_ccc)

# Subset by interaction pathway
lr_res <- lr_res[lr_res$pathway_name %in% pathway_name_thr, ]
lr_res$pathway_name <- factor(lr_res$pathway_name, levels=pathway_name_thr)

# DEA results 

In [ ]:
dea_res_1 <- readRDS("result/dea/scRNAseq/sample_group/IFNAR_fl_CpG_D1_vs_IFNAR_fl_Baseline_D0.rds")[c("cMo", "intMo", "RPM", "cDC2", "Basophil", "MEP", "Proerythroblast", "Erythroblast")]
dea_res_1 <- lapply(dea_res_1, function(x) {list("D1"=x)})

In [ ]:
dea_res_2 <- readRDS("result/dea/scRNAseq/sample_group/IFNAR_fl_CpG_D1_vs_IFNAR_fl_Baseline_D0.rds")[c("cMo", "intMo", "RPM", "cDC2", "Basophil", "MEP", "Proerythroblast", "Erythroblast")]
dea_res_2 <- lapply(dea_res_2, function(x) {list("D3"=x)})

In [ ]:
dea_res <- Map(c, dea_res_1, dea_res_2)

## PTPG B (+/+) vs D1 (+/+) and D3 (+/+)

## cMo

In [ ]:
source <- c("cMo")
target <- c("MEP")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_3/cci_cmo_mep_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_res, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

In [ ]:
source <- c("cMo")
target <- c("Proerythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_3/cci_cmo_proeb_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_res, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

In [ ]:
source <- c("cMo")
target <- c("Erythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_3/cci_cmo_eb_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_res, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

## intMo

In [ ]:
source <- c("intMo")
target <- c("MEP")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_3/cci_intmo_mep_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_res, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

In [ ]:
source <- c("intMo")
target <- c("Proerythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_3/cci_intmo_proeb_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_res, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

In [ ]:
source <- c("intMo")
target <- c("Erythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_3/cci_intmo_eb_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_res, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

## RPM

In [ ]:
source <- c("RPM")
target <- c("MEP")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_3/cci_rpm_mep_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_res, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

In [ ]:
source <- c("RPM")
target <- c("Proerythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_3/cci_rpm_proeb_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_res, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

In [ ]:
source <- c("RPM")
target <- c("Erythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
pdf("result/figures/1_scRNAseq/panel_3/cci_rpm_eb_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_res, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

# Interaction plots

In [ ]:
sources <- c("RPM", "intMo", "cMo")
targets <- c("MEP", "Proerythroblast", "Erythroblast")

In [ ]:
cco_1 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_baseline_d0.rds")
cco_2 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_cpg_d1.rds")
cco_3 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_cpg_d3.rds")
cco_4 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_lysm_cre_baseline_d0.rds")
cco_5 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_lysm_cre_cpg_d1.rds")
cco_6 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_lysm_cre_cpg_d3.rds")

In [ ]:
cco_1 <- subsetCellChat(object=cco_1, idents.use=c(sources, targets))
cco_2 <- subsetCellChat(object=cco_2, idents.use=c(sources, targets))
cco_3 <- subsetCellChat(object=cco_3, idents.use=c(sources, targets))
cco_4 <- subsetCellChat(object=cco_4, idents.use=c(sources, targets))
cco_5 <- subsetCellChat(object=cco_5, idents.use=c(sources, targets))
cco_6 <- subsetCellChat(object=cco_6, idents.use=c(sources, targets))

In [ ]:
levels(cco_1@idents) <- c(targets, sources)
levels(cco_2@idents) <- c(targets, sources)
levels(cco_3@idents) <- c(targets, sources)
levels(cco_4@idents) <- c(targets, sources)
levels(cco_5@idents) <- c(targets, sources)
levels(cco_6@idents) <- c(targets, sources)

In [ ]:
color_use <- color$celltype_low[names(color$celltype_low) %in% c(sources, targets)]

In [ ]:
cco_list <- list(cco_1, cco_2, cco_3, cco_4, cco_5, cco_6)

q_prob_thr <- quantile(c(cco_1@net$prob, cco_2@net$prob, cco_3@net$prob), 0.90)
for (i in seq_along(cco_list)) {
    
    prob_arr <- cco_list[[i]]@net$prob
    prob_arr[prob_arr > q_prob_thr] <- q_prob_thr
    cco_list[[i]]@net$prob <- prob_arr
    
}

In [ ]:
pdf("result/figures/1_scRNAseq/panel_3/cp_cci_net_ctl_cg.pdf", width=2.0, height=2.0)

    replayPlot(netVisual_chord_gene(cco_list[[1]], slot.name="net", sources.use=sources, targets.use=targets, signaling=pathway_name_thr, color.use=color_use, title.name=NULL, show.legend=FALSE, small.gap=1, big.gap=5, lab.cex=0.5))

dev.off()

In [ ]:
pdf("result/figures/1_scRNAseq/panel_3/cp_cci_net_d1_cg.pdf", width=2.0, height=2.0)

    replayPlot(netVisual_chord_gene(cco_list[[2]], slot.name="net", sources.use=sources, targets.use=targets, signaling=pathway_name_thr, color.use=color_use, title.name=NULL, show.legend=FALSE, small.gap=1, big.gap=5, lab.cex=0.5))

dev.off()

In [ ]:
pdf("result/figures/1_scRNAseq/panel_3/cp_cci_net_d3_cg.pdf", width=2.0, height=2.0)

    replayPlot(netVisual_chord_gene(cco_list[[3]], slot.name="net", sources.use=sources, targets.use=targets, signaling=pathway_name_thr, color.use=color_use, title.name=NULL, show.legend=FALSE, small.gap=1, big.gap=5, lab.cex=0.5))

dev.off()